In [1]:
import os
print(os.getcwd())
if os.getcwd().endswith("modelling"):
    os.chdir("..")

from src.modelling import DeepModel_Trainer
from src.augmentation import Transformer,None_Transform
from src.data_modules import DataModule
from torch import nn
import torch
from torchvision import transforms,models

In [6]:
import numpy as np
np.random.choice(np.where(np.array([0,1,0,0,1]) != np.array([0,0,0,0,0]))[0])

1

In [2]:
%env "WANDB_NOTEBOOK_NAME" "ccv1_efficient_net"
%env WANDB_SILENT=True

base_transformer = Transformer(
    None_Transform(), "standard"
).getCompose()
# %%
def base_mlp():
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(
        10000, 1
    )) 
    return model

In [ ]:
efficientnet = DeepModel_Trainer(DataModule(base_transformer), base_mlp)
efficientnet.train_model("Baseline MLP", "base_mlp", num_epochs=6, test_model=False,batchsize_train_data=128)